# Atomic relaxation data

Atomic relaxation data is used to calculate the X-ray and Auger electron spectrum that arises from an ionised atom relaxing back to a lower energy state. Atoms can be ionised through various mechanisms, including but not limited to atomic interactions of photons (photoatomic ionisation), electrons (electroatomic ionisation), etc. When an atom is ionised, a vacancy in one of the atom's electron subshells is created. These vacancies can be filled by the transition of an electron from a higher subshell (which will have a lower binding energy than the subshell with the vacancy). This transition can either be radiative or non-radiative.

In a radiative transition, an electron from another shell fills a vacancy in the current shell while emitting a photon (this electron is often referred to as an X-ray). In a non-radiative transition, an electron from another shell fills a vacancy in the current shell while another electron is emitted (this electron is often referred to as an Auger electron).

Atomic relaxation data in `dryad` is stored within an `AtomicRelaxation` object. The underlying objects inside of an `AtomicRelaxation` object are described here:
- [Transition data](atomic-relaxation-data/1-transition-data.ipynb): `RadiativeTransitionData` and `NonRadiativeTransitionData` to store transition data (shell identifiers, probabilities and optional transition energies)
- [Electron subshell configuration data](atomic-relaxation-data/2-subshell-configuration-data.ipynb): `ElectronSubshellConfiguration` to store transition data and other information for each electron subshell of an atom in the atomic relaxation library

## `AtomicRelaxation`

### Reading atomic relaxation data from a file

An `AtomicRelaxation` instance can currently be created from a ENDF6 or GNDS formatted data files. Reading this type of data from an ACE file will be added at a later time. A number of dedicated static factory functions for each supported format are defined in the `AtomicRelaxation` interface:
- `from_endf_file` for ENDF6 formatted files
- `from_gnds_file` for GNDS 2.0 formatted xml files

The arguements to be provided to these functions are:
- `filename`: the name and path (relative or absolute) of the file
- `normalise`: an option to indicate whether or not to normalise all probability data (default: no normalisation)

Note: the default behaviour in any `dryad` object that has probability data is that a user must explicitly request normalisation at construction or read time.

For example:

In [1]:
from dryad import AtomicRelaxation

oxygen_endf = AtomicRelaxation.from_endf_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf' )
oxygen_gnds = AtomicRelaxation.from_gnds_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf.gnds.xml' )

### Properties

`AtomicRelaxation` contains the atomic relaxation data for a single atom:
- the element identifier for the atom for which the atomci relaxation data is given (`element_identifier`)
- the data for each subshell of the atom (`subshells`)

The `element_identifier` and `subshells` properties are read and write.

The subshell data is sorted by the order given by the electron subshell identifier (i.e. first the K shell, then the L1 shell, etc.). The number of subshells can be retrieved using the `number_subshells` property.

The `AtomicRelaxation` interface also provides a `has_subshell(id)` function to check whether or not a given electron subshell is present and a `subshell(id)` function that can be used to extract a specific subshell (an exception is thrown if the subshell cannot be found).

### Construction

To create an instance of `AtomicRelaxation` the following arguments are required:
- `element`: the element identifier of the atom
- `subshells`: a list of `ElectronSubshellConfiguration` instances with the data for each electron subshell of the atom

and the following argument is optional:
- `normalise`: an option to indicate whether or not to normalise all probability data (default: no normalisation)

For example:

In [2]:
from dryad import AtomicRelaxation
from dryad.atomic import ElectronSubshellConfiguration
from dryad.atomic import RadiativeTransitionData
from dryad.atomic import NonRadiativeTransitionData
from dryad.id import ElementID
from dryad.id import ElectronSubshellID

id = ElementID( 8 )
shells = [ ElectronSubshellConfiguration(
               ElectronSubshellID( 'K' ), 538, 2,
               [ RadiativeTransitionData( ElectronSubshellID( 'L2' ), 0.00190768 ),
                 RadiativeTransitionData( ElectronSubshellID( 'L3' ), 0.00380027 ) ],
               [ NonRadiativeTransitionData( ElectronSubshellID( 'L1' ), ElectronSubshellID( 'L1' ), 0.178644 ),
                 NonRadiativeTransitionData( ElectronSubshellID( 'L1' ), ElectronSubshellID( 'L2' ), 0.116224 ),
                 NonRadiativeTransitionData( ElectronSubshellID( 'L1' ), ElectronSubshellID( 'L3' ), 0.230418 ),
                 NonRadiativeTransitionData( ElectronSubshellID( 'L2' ), ElectronSubshellID( 'L2' ), 0.0110822 ),
                 NonRadiativeTransitionData( ElectronSubshellID( 'L2' ), ElectronSubshellID( 'L3' ), 0.291115 ),
                 NonRadiativeTransitionData( ElectronSubshellID( 'L3' ), ElectronSubshellID( 'L3' ), 0.166809 ) ] ),
        ElectronSubshellConfiguration( ElectronSubshellID( 'L1' ), 28.48, 2 ),
        ElectronSubshellConfiguration( ElectronSubshellID( 'L2' ), 13.62, 1.33 ),
        ElectronSubshellConfiguration( ElectronSubshellID( 'L3' ), 13.62, 2.67 ) ]

relaxation = AtomicRelaxation( element = id, subshells = shells, normalise = False )

### Calculate transition energies

Since the transition energy can be calculated using the binding energy of the current subshell and the originating subshell (see [Transition data](atomic-relaxation-data/1-transition-data.ipynb) for more information), the transition energy is considered a redundant quantity. As such, atomic relaxation data read from a GNDS file does not populate this quantity. These can be calculated by calling the `calculate_transition_energies()` function on an `AtomicRelaxation` instance.

### Normalisation of transition probabilities

As indicated above, when constructing or reading atomic relaxation data, the user has the possibility of normalising the probability data (the `normalise` option) so that the sum of all transition probabilities for each electron subshell is 1 (if the subshell has transition data). The `normalise()` allows a user to do the same after the `AtomicRelaxation` instance was created.

If the user changes one or more transition probabilities, this function can then be used to renormalise the data. It is the responsibility of the user to request this renormalisation whenever required.

### Example

In this example, we open atomic relaxation data for oxygen from both an ENDF and GNDS file. As provided in the ENDF data, the transition energy values are not consistent with the binding energies of the other shells in the Oxygen relaxation data from ENDF/B-VIII.1 so they need to be recalculated. The GNDS data does not provide the transition energies since these values are redundant (as they can be calculated from the binding energies of the subshells involved in the transitions. We will therefore recalculate these transition energies.

In addition to the updates to the transition energies, we also normalise the probability data for all transitions and then compare the values between the ENDF and GNDS data.

In [ ]:
import math
from dryad import AtomicRelaxation

oxygen_endf = AtomicRelaxation.from_endf_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf' )
oxygen_gnds = AtomicRelaxation.from_gnds_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf.gnds.xml' )

oxygen_endf.calculate_transition_energies()
oxygen_gnds.calculate_transition_energies()

oxygen_endf.normalise()
oxygen_gnds.normalise()

def print_subshell_data( endf, gnds ) :

    print( '  Subshell identifier             : {}'.format( endf.identifier.name ) )
    print( '  Number radiative transitions    : {}'.format( endf.number_radiative_transitions ) )
    print( '  Number non-radiative transitions: {}'.format( endf.number_non_radiative_transitions ) )

    print( '' )
    print( '              Quantity            |       ENDF      |       GNDS      |   Equal?' )
    print( '  ' + '- ' * 41 )
    print( '  Binding energy                  |     {:6.2f}      |     {:6.2f}      |    {}'
           .format( endf.binding_energy, gnds.binding_energy,
                    math.isclose( endf.binding_energy, gnds.binding_energy ) ) )
    print( '  Population                      |     {:6.2f}      |     {:6.2f}      |    {}'
           .format( endf.population, gnds.population,
                    math.isclose( endf.population, gnds.population ) ) )

    if endf.has_transitions :

        print( '  Total radiative probability     |  {:1.11f}  |  {:1.11f}  |    {}'
               .format( endf.total_radiative_probability, gnds.total_radiative_probability,
                        math.isclose( endf.total_radiative_probability, gnds.total_radiative_probability ) ) )
        print( '  Total non-radiative probability |  {:1.11f}  |  {:1.11f}  |    {}'
               .format( endf.total_non_radiative_probability, gnds.total_non_radiative_probability,
                        math.isclose( endf.total_non_radiative_probability,
                                      gnds.total_non_radiative_probability ) ) )

        print( '' )
        print( '    Transition energy  |   ENDF   |   GNDS   |   Equal?' )
        print( '  ' + '- ' * 30 )
        for index in range( endf.number_radiative_transitions ) :

            left = endf.radiative_transitions[index].energy
            right = gnds.radiative_transitions[index].energy
            print( '        Radiative      |  {:1.2f}  |  {:1.2f}  |    {}'
                   .format( left, right, math.isclose( left, right ) ) )

        for index in range( endf.number_non_radiative_transitions ) :

            left = endf.non_radiative_transitions[index].energy
            right = gnds.non_radiative_transitions[index].energy
            print( '      Non-radiative    |  {:1.2f}  |  {:1.2f}  |    {}'
                   .format( left, right, math.isclose( left, right ) ) )

        print( '' )
        print( '    Transition prob.   |       ENDF      |       GNDS      |   Equal?' )
        print( '  ' + '- ' * 35 )
        for index in range( endf.number_radiative_transitions ) :

            left = endf.radiative_transitions[index].probability
            right = gnds.radiative_transitions[index].probability
            print( '        Radiative      |  {:1.11f}  |  {:1.11f}  |    {}'
                   .format( left, right, math.isclose( left, right ) ) )

        for index in range( endf.number_non_radiative_transitions ) :

            left = endf.non_radiative_transitions[index].probability
            right = gnds.non_radiative_transitions[index].probability
            print( '      Non-radiative    |  {:1.11f}  |  {:1.11f}  |    {}'
                   .format( left, right, math.isclose( left, right ) ) )

    print( '' )

for index in range( oxygen_endf.number_subshells ) :

    print_subshell_data( oxygen_endf.subshells[index], oxygen_gnds.subshells[index] )

  Subshell identifier             : K
  Number radiative transitions    : 2
  Number non-radiative transitions: 6

              Quantity            |       ENDF      |       GNDS      |   Equal?
  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
  Binding energy                  |     538.00      |     538.00      |    True
  Population                      |       2.00      |       2.00      |    True
  Total radiative probability     |  0.00570794914  |  0.00570794914  |    True
  Total non-radiative probability |  0.99429205086  |  0.99429205086  |    True

    Transition energy  |   ENDF   |   GNDS   |   Equal?
  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
        Radiative      |  524.38  |  524.38  |    True
        Radiative      |  524.38  |  524.38  |    True
      Non-radiative    |  481.04  |  481.04  |    True
      Non-radiative    |  495.90  |  495.90  |    True
      Non-radiative    |  495.90  |  495.90  |    True
    

### Usage notes

Only properties can be used for modifying the data. If a user wishes to replace subshell data as a whole, the `subshells` property should be used to replace all data. Indexing into a particular subshell object and then replacing the entire object does NOT work. No exception is raised, but the data is NOT changed as is shown here:

In [4]:
print( 'Subshell identifier: {}'.format( relaxation.subshells[0].identifier.name ) )
print( 'Binding energy     : {}'.format( relaxation.subshells[0].binding_energy ) )
print( 'Population         : {}'.format( relaxation.subshells[0].population ) )

relaxation.subshells[0] = ElectronSubshellConfiguration( ElectronSubshellID( 'L1' ), 28.48, 2 )

print( 'Subshell identifier: {}'.format( relaxation.subshells[0].identifier.name ) )
print( 'Binding energy     : {}'.format( relaxation.subshells[0].binding_energy ) )
print( 'Population         : {}'.format( relaxation.subshells[0].population ) )

Subshell identifier: K
Binding energy     : 538.0
Population         : 2.0
Subshell identifier: K
Binding energy     : 538.0
Population         : 2.0
